In [ ]:
# This file is aimed to test the 2D Ising model with four body interaction terms using SLMC method.

#import libraries
import numpy as np
import numpy.random as rnd
import matplotlib.pyplot as plt
from Configuration import Configuration
from Hamiltonian import first_NN_interaction, second_NN_interaction, third_NN_interaction
from LocalUpdate import LocalUpdate
from SelfLearningUpdate import SelfLearningUpdate
import itertools
from sklearn import linear_model

In [ ]:
# transfer spins matrix to image 
def config_to_image(spins):
    """Turn a spins array into an image"""
    L = len(spins)
    im = np.zeros([L,L,3])
    for i,j in itertools.product(range(L), repeat=2):
        im[i,j,:] = (1,0,0) if spins[i,j]==1 else (0,0,0)
    return im

In [ ]:
def Make_Samples_Local(size, J, K, T, Nsamples, Nsteps):
    """Generate samples from Local Update Method.
    Input parameters: 
    size: size of the lattice;
    J, K: parameters of the Hamiltonian;
    T: temperature;
    Nsamples: the number of the samples;
    Nsteps: steps taken in MC simulation to reach equilibrium;
    Output:
    A list of samples, each sample has the form: [1st NN interaction sum, 2nd NN interaction sum, 3rd NN interaction sum, energy]
    """
    #initiate sample list
    samples = []
    
    for n in range(Nsamples):
        spins = rnd.choice([-1, 1], size=(size, size))  # either +1 or -1
        for i in range(Nsteps):
            for k in range(size * size):
                update = LocalUpdate(spins, J, K, T)
                spins = update.local_update()
        config = Configuration(spins, size, J, K, T)
            
        E1 = first_NN_interaction(spins)
        E2 = second_NN_interaction(spins)
        E3 = third_NN_interaction(spins)
        Energy = config.energy
        samples.append([Energy, E1, E2, E3])
        # print info because progress can be slow
        print("Sample %.0f: %.2f percent done."%(n+1, 100*(n+1)/Nsamples))
    return samples


def Make_Samples_SelfLearning(size, J, K, T, Nsamples, Nsteps, eff_param):
    """Generate samples from Local Update Method.
    Input parameters: 
    size: size of the lattice;
    J, K: parameters of the Hamiltonian;
    T: temperature;
    Nsamples: the number of the samples;
    Nstep: steps taken in MC simulation to reach equilibrium;
    eff_param: parameters of effective Hamiltonian.
    Output:
    A list of samples, each sample has the form: [energy, 1st NN interaction sum, 2nd NN interaction sum, 3rd NN interaction sum]
    """
    #initiate sample list
    samples = []
    
    for n in range(Nsamples):
        spins = rnd.choice([-1, 1], size=(size, size))  # either +1 or -1
        for i in range(Nsteps):
            update = SelfLearningUpdate(spins, J, K, T, eff_param)
            spins = update.SLMC_Update()
        config = Configuration(spins, size, J, K, T)
            
        E1 = first_NN_interaction(spins)
        E2 = second_NN_interaction(spins)
        E3 = third_NN_interaction(spins)
        Energy = config.energy
        samples.append([Energy, E1, E2, E3])
        # print info because progress can be slow
        print("Sample %.0f: %.2f percent done."%(n+1, 100*(n+1)/Nsamples))
    return samples

         
            

In [ ]:
def train_eff_Hamil(samples, n):
    """Train effective Hamiltonian from the samples.
    Input parameters:
    samples: a list of samples, each sample has the form: [energy, 1st NN interaction sum, 2nd NN interaction sum, 3rd NN interaction sum]
    n: the order of interactions that is considered in H_eff.
    Output:
    eff_param: parameters of effective Hamiltonian, a list with the first term being E0 and next terms being J coefficients
    """
    eff_param = []
    samples = np.array(samples)
    energy = samples[:, 0:1]
    interaction = samples[:,1:n+1]
    
    #use linear model to get E0 and Js
    reg = linear_model.LinearRegression()
    reg.fit(interaction, energy)
    
    coef = reg.coef_
    eff_param = np.append(reg.intercept_, coef)
    return eff_param

In [ ]:
# T > Tc, train some samples using Local Update Method, T = 5
L = 20
J = 1.
K = 0.2
T = 5
Nsamples = 50 # modified later
Nsteps = 10000 # trail number, modified later
n = 1 # take only one J coefficient
samples = Make_Samples_Local(L, J, K, T, Nsamples, Nsteps)
print(samples)
eff_param = train_eff_Hamil(samples, n)
print(eff_param)


In [ ]:
# T > Tc, train some samples using Local Update Method, T = 5
L = 20
J = 1.
K = 0.2
T = 5
Nsamples = 50 #modified later
Nsteps = 10000 # trail number, modified later
n = 3 # take 3 J coefficient
samples = Make_Samples_Local(L, J, K, T, Nsamples, Nsteps)
print(samples)
eff_param = train_eff_Hamil(samples, n)
print(eff_param)

In [ ]:
# T = Tc, use the effective Hamiltonian to get more samples, and reobtain effective Hamiltonian iteratively.
# We will get optimized effective Hamiltonian in the end.
L  = 20
J = 1.
K = 0.2
T = 2.5
Nsamples = 5
Nsteps = 10000
n = 1

# Set iteration step
Iter = 5 # modified later
for k in range(Iter):
    samples = Make_Samples_Self_Learning(L, J, K, T, Nsamples, Nsteps, eff_param)
    eff_param = train_eff_Hamil(samples, n)
    print(eff_param)

In [ ]:
# Use the optimized effective Hamiltonian in SLMC update

# Magnetization vs Step
L = 20
J = 1
K = 0.2
T = 2.5

spins = rnd.choice([-1,1],size = (L, L))

import time
t1 = time.time()
n_cycles = 1000

mr = np.zeros(n_cycles)

# Monte Carlo
for n in range(n_cycles):
    update = SelfLearningUpdate(spins, J, K, T, eff_param)
    spins = update.SLMC_Update()
    config = Configuration(spins, L, J, K, T)
    mr[n] = abs(config.magnetization)
mr /= float(L**2)
t2 = time.time()

print(t2-t1)
plt.plot(mr, '-')
plt.title("Magnetization", fontsize=25)
plt.xlabel("$t$", fontsize=20)
plt.ylabel("$m$", fontsize=20)
plt.ylim(-1,1)

In [ ]:
# Physical Quantity vs Temp
import time
# This is a production script, it will save the results in files
L = 20
J = 1
K = 0.2

spins = rnd.choice([-1,1],size = (L, L))

# set temperature range
temp_range = np.hstack([np.arange(0.5,2.0,0.5), np.arange(2.0,2.7,0.1), np.arange(2.7,3.3,0.2)])
mag = np.zeros_like(temp_range)
energ = np.zeros_like(temp_range)
chi = np.zeros_like(temp_range)
cv = np.zeros_like(temp_range)

time1 = time.time()
# lattice size
l = [20]
for L in l:
    for i,T in enumerate(temp_range):
    
        av_m, av_m2, av_e, av_e2 = 0,0,0,0

        n_cycles = 10000
        length_cycle = L*L
        n_warmup = 300

        # Monte Carlo
        for n in range(n_warmup+n_cycles):
            update = SelfLearningUpdate(spins, J, K, T, eff_param)
            spins = update.SLMC_Update()
                
            if n >= n_warmup:
                config = Configuration(spins, L, J, K, T)
                av_e  += config.energy
                av_e2 += config.energy**2
                av_m  += config.magnetization
                av_m2 += config.magnetization**2
            
        # normalize averages
        av_m  /= float(n_cycles)
        av_m2 /= float(n_cycles)
        av_e  /= float(n_cycles)
        av_e2 /= float(n_cycles)
            
        # get physical quantities
        fact = 1./L**2
        mag[i] = fact * av_m
        energ[i] = fact * av_e
        cv[i] = fact * (av_e2 - av_e**2) / T**2
        chi[i] = fact * (av_m2 - av_m**2) / T
    
        # print info because progress can be slow
        print("T = %f and %.2f percent done"%(T, (100.*(i+1))/len(temp_range)))


    # save quantities in a file
    np.savetxt("SLMC_update_test_energ_%i.dat"%L, energ)
    np.savetxt("SLMC_update_test_mag_%i.dat"%L, mag)
    np.savetxt("SLMC_update_test_cv_%i.dat"%L, cv)
    np.savetxt("SLMC_update_test_chi_%i.dat"%L, chi)
    
time2 = time.time()
print('time: %i'%(time2-time1))

for L in l:
    energ = np.loadtxt("SLMC_update_test_energ_%i.dat"%L)
    mag = np.loadtxt("SLMC_update_test_mag_%i.dat"%L)
    cv = np.loadtxt("SLMC_update_test_cv_%i.dat"%L)
    chi = np.loadtxt("SLMC_update_test_chi_%i.dat"%L)
    
    fig = plt.figure()
    plt.plot(temp_range, np.loadtxt("SLMC_update_test_energ_%i.dat"%L), '-o', label="energy")
    plt.plot(temp_range, np.abs(np.loadtxt("SLMC_update_test_mag_%i.dat"%L)), '-o', label="magnetization")
    plt.plot(temp_range, np.loadtxt("SLMC_update_test_cv_%i.dat"%L), '-o', label="specific heat")
    plt.plot(temp_range, np.loadtxt("SLMC_update_test_chi_%i.dat"%L)/1000, '-o', label="susceptibility")

#plt.plot([Tc,Tc], [0,1.6], '--', lw=3)
    plt.legend()
    plt.title("Physical quantities", fontsize=25)
    plt.xlabel("$T$", fontsize=20)
    plt.show()

In [ ]:
# autocorrelation fitting

L = 20
J = 1
K = 0.2
T = 2.5

nt = 300
av_m = 0.0
av_correl = np.zeros([nt])
previous_m = []

n_cycles = 10000
n_warmup = 1000

# Monte Carlo
for n in range(n_cycles + n_warmup + nt):
    
    update = SelfLearningUpdate(spins, J, K, T, eff_param)
    spins = update.Wolff_Update_1()
            
    if n >= n_warmup:
        config = Configuration(spins, L, J, K, T)
        previous_m.insert(0, config.magnetization)
        #print(previous_m)
        # get rid of furthest previous m
        if n >= n_warmup+nt:
            previous_m.pop()
            #print(previous_m)
            av_m += config.magnetization
            #print(av_m)
            for k in range(nt):
                av_correl[k] += previous_m[k] * config.magnetization
            #print(av_correl)
        
av_m  /= float(n_cycles)
av_correl /= float(n_cycles)


In [ ]:
from scipy.optimize import curve_fit

n_fit_pts = 100
xr = np.arange(n_fit_pts, dtype=float)

# fit autocorrelation function
f = lambda x, a, b: a*np.exp(-x/float(b))
a, b = curve_fit(f, xr, (av_correl-av_m**2)[0:n_fit_pts], p0=(1000,1))[0]
print("Autocorrelation time =", b)

plt.plot(np.log(np.abs(av_correl-av_m**2)), '-ro', lw=2, alpha=0.5)

plt.plot(xr, np.log(f(xr, a, b)), 'b-', lw=2)
plt.plot([0,100], [0,0], 'k--', lw=2)

plt.title("Autocorrelation function", fontsize=25)
plt.xlabel("$t$", fontsize=20)
plt.ylabel(r"$\mathcal{C}(t)$", fontsize=20)
#plt.xlim(0, n_fit_pts+10)